In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("/Users/jun/FullStack_GPT/files/IS20_Erosion_ErosionCorrosion v1.8.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retrieval = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retrieval,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("Describe the critical factor of erosion")

AIMessage(content='The critical factors of erosion are the solids content, velocity, corrosiveness of the process stream, and material composition. These factors determine the degree of erosion and erosion-corrosion that a material may experience. The solids content in the process fluid significantly impacts the degree of erosion and erosion-corrosion. Higher velocities in the process stream impart higher energy on the metal surface, damaging and removing protective layers and exposing fresh metal to corrosion. The corrosiveness of the process stream also plays a role, as more aggressive fluids lead to more rapid corrosion. Finally, the material composition of the exposed material affects its susceptibility to erosion and erosion-corrosion.')